In [138]:
# pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import lyricsgenius
from spotipy.oauth2 import SpotifyClientCredentials

# Spotify API credentials
SPOTIPY_CLIENT_ID = ''
SPOTIPY_CLIENT_SECRET = ''
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
# Initialize the Spotify API client
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=40) #spotify object to access API

# Create a Genius API client
GENIUS_CLIENT_ACCESS_TOKEN = ''
genius = lyricsgenius.Genius(GENIUS_CLIENT_ACCESS_TOKEN, timeout=40)





In [140]:
def lyrics(song, artist):
    genius = lyricsgenius.Genius('lOa1BiaTot1bMp9nwzBq3CaTUrIKAYLJmu9u0e0npR7_zzplerMV_IgtO4gPgXcY')
    song = genius.search_song(title=song, artist=artist)
    try:
      lyrics = song.lyrics
      print(lyrics)
      print('\n')
    except AttributeError:
      print("There are no lyrics for this song on Genius")
      print('\n')

In [141]:
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ', '-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ', '-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/' + artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

In [142]:
def lyrics_onto_frame(df1, artist_name):
    for i, x in enumerate(df1['Track Name']):
        test = scrape_lyrics(artist_name, x)
        print(test)
        df1.loc[i, 'Lyrics'] = test
    return df1

In [143]:
import re
artist_names = []
track_names = []
release_dates = []
genres = []
lyrics = []

pop_tracks = sp.search(q='genre:retro', type='track', limit=50, offset=80)

# Iterate through the search results and collect data
i = 1
for track in pop_tracks['tracks']['items']:
    try:
        print(i)
        i += 1

        song = genius.search_song(track['name'], track['artists'][0]['name'])
        song = song.to_dict()
        l = song["lyrics"]
        l = re.sub(r'^.*?\n+', '', l, count=1)
        l = re.sub(r'\s+', ' ', l)

        lyrics.append(l)
        artist_names.append(track['artists'][0]['name'])
        track_names.append(track['name'])
        release_dates.append(track['album']['release_date'])
        genres.append('retro')

    except:
        pass


1
Searching for "Where You're At" by Allen Stone...
Done.
2
Searching for "OCT 33" by Black Pumas...
Done.
3
Searching for "i'm confident that i'm insecure (acoustic-ish)" by Lawrence...
No results found for: 'i'm confident that i'm insecure (acoustic-ish) Lawrence'
4
Searching for "Good To You" by Jonny P...
Done.
5
Searching for "Freckles" by Lawrence...
Done.
6
Searching for "Louie, Louie" by Los Yesterdays...
Done.
7
Searching for "Stayin' Alive" by Scary Pockets...
Done.
8
Searching for "Time" by Los Yesterdays...
Done.
9
Searching for "Sea Star" by St. Paul & The Broken Bones...
Done.
10
Searching for "If You Don't Want My Love" by Jalen Ngonda...
Done.
11
Searching for "100 Days, 100 Nights" by Sharon Jones & The Dap-Kings...
Done.
12
Searching for "Sanctify" by St. Paul & The Broken Bones...
Done.
13
Searching for "Broken Bones and Pocket Change" by St. Paul & The Broken Bones...
Done.
14
Searching for "Vacation" by The Dip...
Done.
15
Searching for "Misty Morning" by Lawrence.

In [144]:
# for track in pop_tracks['tracks']['items']:
#     artist_name = track['artists'][0]['name']
#     track_name = track['name']

#     # Search for lyrics using the artist name and track name
#     song = genius.lyrics(song_url=track['external_urls'])

#     if song:
#         print(song)
#     else:
#         lyrics.append("Lyrics not found")

In [145]:

# Create a DataFrame
data = {
    'artist_name': artist_names,
    'track_name': track_names,
    'release_date': release_dates,
    'genre': genres,
    'lyrics': lyrics
}
import pandas as pd
df = pd.DataFrame(data)

df['release_date'] = pd.to_datetime(df['release_date'])
df['release_date'] = df['release_date'].dt.year.astype('int64')

In [146]:
df

,artist_name,track_name,release_date,genre,lyrics
0,Allen Stone,Where You're At,2021,retro,I keep my dirt on the surface so you don't got...
1,Black Pumas,OCT 33,2019,retro,"I've got your number, lonely October 33 I wear..."
2,Jonny P,Good To You,2016,retro,"It's been a long, long time Since I saw you, y..."
3,Lawrence,Freckles,2021,retro,(You know you're beautiful!) [Verse 1] I think...
4,Los Yesterdays,"Louie, Louie",2023,retro,* 07/01 : Despised Icon - Consumed By Your Poi...
5,Scary Pockets,Stayin' Alive,2023,retro,"Well, you can tell by the way I use my walk I'..."
6,Los Yesterdays,Time,2020,retro,"Just in, Pigeon John and Flynn Adam Atkins arr..."
7,St. Paul & The Broken Bones,Sea Star,2023,retro,Deep in the ocean’s groove It used to be green...
8,Jalen Ngonda,If You Don't Want My Love,2023,retro,"In this night, a starry sky My baby told me wi..."
9,Sharon Jones & The Dap-Kings,"100 Days, 100 Nights",2007,retro,"To know a man's heart 100 days, 100 nights To ..."


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_name   48 non-null     object
 1   track_name    48 non-null     object
 2   release_date  48 non-null     int64 
 3   genre         48 non-null     object
 4   lyrics        48 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.0+ KB


In [ ]:
df.to_csv("Student_dataset_3.csv", index=False)